In [5]:
import random
#random.seed(99)
import pandas as pd
import numpy as np
import dask.dataframe as dd
import seaborn as sns
import dask.datasets
from dask_sql import Context
import datetime
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = None

# Подсчет количества пользователей и запросов по категории на одном файле

In [2]:
def agg_df(df, year):
    start_dt = datetime.datetime.now()
    print('Start:', start_dt)
    df1 = df[['user', 'rubricName', 'date']]
    
    try:
        pd_df1_req_cnt = pd.read_excel(f'df1_req_cnt_{year}.xlsx')
        pd_df1_user_cnt = pd.read_excel(f'df1_user_cnt_{year}.xlsx')
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_index=True, right_index=True)
    except:
        df1_user_cnt = df1.groupby('rubricName').user.nunique().compute()
        df1_req_cnt = df1.groupby('rubricName').date.count().compute()
        df1_req_cnt.to_excel(f'df1_req_cnt_{year}.xlsx')
        df1_user_cnt.to_excel(f'df1_user_cnt_{year}.xlsx')
        pd_df1_req_cnt = pd.DataFrame(df1_req_cnt)
        pd_df1_user_cnt = pd.DataFrame(df1_user_cnt)
        df1_agg = pd_df1_req_cnt.merge(pd_df1_user_cnt, left_on='rubricName', right_on='rubricName')
    
    df1_agg = df1_agg.sort_values('date', ascending=False)
    df1_agg['rubricName'] = df1_agg['rubricName_x']
    df1_agg = df1_agg.set_index('rubricName')[['date', 'user']].fillna(0).astype(int)
    df1_agg.to_excel(f'df1_agg_{year}.xlsx', index=False)
    end_dt = datetime.datetime.now()
    print('End:', end_dt)
    print('Duration:', end_dt-start_dt)
    return df1_agg

### Москва

In [3]:
df20_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'})
df20_agg = agg_df(df20_raw, 2020)
df21_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'})
df21_agg = agg_df(df21_raw, 2021)
df22_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'})
df22_agg = agg_df(df22_raw, 2022)
df23_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'})
df23_agg = agg_df(df23_raw, 2023)

Start: 2023-03-13 16:27:50.341337
End: 2023-03-13 16:27:53.322987
Duration: 0:00:02.981650
Start: 2023-03-13 16:27:53.373402
End: 2023-03-13 16:27:53.922772
Duration: 0:00:00.549370
Start: 2023-03-13 16:27:53.970086
End: 2023-03-13 16:27:54.445928
Duration: 0:00:00.475842
Start: 2023-03-13 16:27:54.524908
End: 2023-03-13 16:27:55.025557
Duration: 0:00:00.500649


### СПБ

In [21]:
spb20_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2020_974783.csv")
spb20_agg = agg_df(spb20_raw, 'spb_2020')

Start: 2023-03-13 17:56:43.702246
End: 2023-03-13 17:56:43.902273
Duration: 0:00:00.200027


In [ ]:
spb21_raw = dd.read_csv(r"C:\Users\Mamed\Downloads\СПБ_2021_974784.csv")
spb21_agg = agg_df(spb21_raw, 'spb_2021')

# Подсчет количества пользователей по всем файлам

### Все категории

In [70]:
#df_all = dd.concat([df20_raw,df21_raw,df22_raw,df23_raw])

#df_all.to_parquet('df_all')

df_all = dd.read_parquet('df_all')

users_itogo = df_all.groupby('rubricName').user.nunique().compute()

users_itogo_result = df_all.user.nunique().compute()

df20_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'})
df21_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'})
df22_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'})
df23_raw = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'})

df20_result_cnt = df20_raw.user.nunique().compute()
df21_result_cnt = df21_raw.user.nunique().compute()
df22_result_cnt = df22_raw.user.nunique().compute()
df23_result_cnt = df23_raw.user.nunique().compute()

print(users_itogo_result, 
df20_result_cnt,
df21_result_cnt,
df22_result_cnt,
df23_result_cnt,)

users_itogo.to_excel('users_itogo.xlsx')

5512385 1926180 1624863 1920044 1947122


### Часть категорий

In [80]:
rubrics = pd.read_excel(r"C:\Users\Mamed\Downloads\Свод_2ГИС_Москва_230311_0145.xlsx")[['Рубрика']]
df_all = dd.read_parquet('df_all')
df_filtered = df_all.merge(rubrics, left_on='rubricName', right_on='Рубрика')

df_filtered.to_parquet('df_filtered')

users_filtered_itogo = df_filtered.groupby('rubricName').user.nunique().compute()
users_filtered_itogo_result = df_filtered.user.nunique().compute()
users_filtered_itogo.to_excel('users_filtered_itogo.xlsx')

df20_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2020_969792.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')
df21_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2021_970748.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')
df22_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2022_970753.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')
df23_filtered = dd.read_csv(r"C:/Users/Mamed/Downloads/Москва_2023_970754.csv", dtype={'fl': 'object'}).merge(rubrics, left_on='rubricName', right_on='Рубрика')

df20_filtered_result_cnt = df20_filtered.user.nunique().compute()
df21_filtered_result_cnt = df21_filtered.user.nunique().compute()
df22_filtered_result_cnt = df22_filtered.user.nunique().compute()
df23_filtered_result_cnt = df23_filtered.user.nunique().compute()

print(users_filtered_itogo_result, 
df20_filtered_result_cnt,
df21_filtered_result_cnt,
df22_filtered_result_cnt,
df23_filtered_result_cnt,)

4459161 1627743 1305787 1536122 1536886


# Попытка посчитать рейтинги автоматически

In [34]:
df_all_agg = (df20_agg
     .merge(df21_agg, left_index=True, right_index=True, how='outer', suffixes=('_20', '_21'))
     .merge(df22_agg, left_index=True, right_index=True, how='outer')
     .merge(df23_agg, left_index=True, right_index=True, how='outer', suffixes=('_22', '_23'))
).fillna(0).astype(int)

df_all_agg['date_all'] = df_all_agg.date_20 + df_all_agg.date_21 + df_all_agg.date_22 + df_all_agg.date_23
df_all_agg['date_all_prc'] = (df_all_agg['date_all'] / df_all_agg['date_all'].sum() * 100)

df_all_agg = df_all_agg[['user_20', 'user_21', 'user_22','user_23', 'date_20', 'date_21', 'date_22'
                         , 'date_23', 'date_all', 'date_all_prc']].sort_values('date_all', ascending=False)

df_all_agg.columns = ['users_20', 'users_21', 'users_22','users_23',
                      'requests_20', 'requests_21', 'requests_22', 'requests_23', 'requests_all', 'requests_all_prc']

df_all_agg = df_all_agg.reset_index()

#df_all_agg.to_excel('Свод_2ГИС_Москва_230310_2145.xlsx', index=True)

df_all_agg_short = df_all_agg.head(100)

u_cols_list = [20,21,22,23]

for i in u_cols_list:
    for j in u_cols_list:
        if i > j:
            pass #print(f'df_all_agg_short["users_delta_{i}_{j}"] = df_all_agg_short["users_{i}"] - df_all_agg_short["users_{j}"]')

df_all_agg_short["users_delta_21_20"] = df_all_agg_short["users_21"] - df_all_agg_short["users_20"]
df_all_agg_short["users_delta_22_20"] = df_all_agg_short["users_22"] - df_all_agg_short["users_20"]
df_all_agg_short["users_delta_22_21"] = df_all_agg_short["users_22"] - df_all_agg_short["users_21"]
df_all_agg_short["users_delta_23_20"] = df_all_agg_short["users_23"] - df_all_agg_short["users_20"]
df_all_agg_short["users_delta_23_21"] = df_all_agg_short["users_23"] - df_all_agg_short["users_21"]
df_all_agg_short["users_delta_23_22"] = df_all_agg_short["users_23"] - df_all_agg_short["users_22"]

df_all_agg_short["requests_delta_21_20"] = df_all_agg_short["requests_21"] - df_all_agg_short["requests_20"]
df_all_agg_short["requests_delta_22_20"] = df_all_agg_short["requests_22"] - df_all_agg_short["requests_20"]
df_all_agg_short["requests_delta_22_21"] = df_all_agg_short["requests_22"] - df_all_agg_short["requests_21"]
df_all_agg_short["requests_delta_23_20"] = df_all_agg_short["requests_23"] - df_all_agg_short["requests_20"]
df_all_agg_short["requests_delta_23_21"] = df_all_agg_short["requests_23"] - df_all_agg_short["requests_21"]
df_all_agg_short["requests_delta_23_22"] = df_all_agg_short["requests_23"] - df_all_agg_short["requests_22"]

df_all_agg_short = df_all_agg_short.reset_index()

# Алгоритм сокращения топ-мест в рейтингах для категорий

In [838]:
original_svod = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Свод_2ГИС_Москва_230313_2115.xlsx")

In [839]:
svod = original_svod[['Рубрика', 'UserMaxTotal', 'UserMaxFirstYear',
       'UserMaxLastYear', 'UserMinLastYear', 'UserMinFirstYear',
       'UserJustGrowth', 'UserJustFall', 'UserMax2021', 'UserMax2022',
       'UserMin2021', 'UserMin2022', 'UserTBTB', 'UserBTBT', 'UserStable',
       'PositiveCorrelationUserReq', 'ReqMaxTotal', 'ReqMaxFirstYear',
       'ReqMaxLastYear', 'ReqMinLastYear', 'ReqMinFirstYear', 'ReqJustGrowth',
       'ReqJustFall', 'ReqMax2021', 'ReqMax2022', 'ReqMin2021', 'ReqMin2022',
       'ReqTBTB', 'ReqBTBT', 'ReqStable', 'NegativeCorrelationUserReq',
       'InLists', 'inList, сумма'
            ]]

new_svod = (
    svod
    #.loc[svod['InLists'] > 1]
    .fillna(0)
    .sort_values('inList, сумма', ascending=False)
    .set_index('Рубрика')
)

new_svod['cnt_horizontal'] = new_svod['InLists']

In [890]:
def horizontal_optimization(table_to_optimize):
    new_svod = table_to_optimize
    test_svod = new_svod.drop(['InLists', 'inList, сумма'], axis=1).copy()
    index_set = set(new_svod.index)
    cnt = 0
    total_sum = 9999
    while cnt < 10000 and total_sum > 5141 and len(index_set)> 0:
        j = index_set.pop()
        cnt += 1
        cols = [i for i in test_svod.loc[j][test_svod.loc[j] > 0].sort_values(ascending=False).index]
        for i in cols:   
            cnt_horizontal = test_svod.loc[j][test_svod.loc[j] > 0].count()
            cnt_vertical = test_svod[test_svod[i] > 0][i].count()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
        
        cols_set = set(cols)
        cnt2 = 0
        while cols_set:
            i = cols_set.pop()
            cnt_horizontal = test_svod.loc[j][test_svod.loc[j] > 0].count()
            cnt_vertical = test_svod[test_svod[i] > 0][i].count()
            if cnt_vertical > 15 and cnt_horizontal > 1:
                test_svod[i][j] = 0
        total_sum = test_svod.sum(axis=1).sum()
        
        #print(total_sum)
    test_svod['cnt_horizontal'] = test_svod.drop('cnt_horizontal', axis=1).replace(range(1, 26), 1).sum(axis=1)
    test_svod = test_svod.sort_values('cnt_horizontal', ascending=False)
    return test_svod

In [891]:
def vertical_optimization(table_to_optimize, original_table):
    df = table_to_optimize
    new_svod = original_table
    for i in range(20):
        for cat_to_optimize in df.index:
            # сейчас есть значение, превращаем в ноль
            col = df.drop('cnt_horizontal', axis=1).loc[cat_to_optimize].sort_values(ascending=False).index[0]

            # сейчас 0 возвращаем значение из бэкапа
            try:
                cat_to_reset = (
                     df
                     .loc[df.index != cat_to_optimize]
                     .loc[(new_svod.sort_index().loc[:, col] != df.sort_index().loc[:, col]) 
                          & (df[col] == 0)]
                     .loc[:,[col, 'cnt_horizontal']]
                     .sort_values([col, 'cnt_horizontal'], ascending=[False, True])
                     .index[0]
                )
                if (df.loc[cat_to_reset, 'cnt_horizontal'] + 1 < df.loc[cat_to_optimize, 'cnt_horizontal']
                   ):
                    df.loc[cat_to_optimize, col] = 0
                    df.loc[cat_to_reset, col] = new_svod.loc[cat_to_reset, col]
                    df['cnt_horizontal'] = df.drop('cnt_horizontal', axis=1).replace(range(1, 26), 1).sum(axis=1)
            except:
                pass
    return df

In [892]:
new_svod.value_counts('cnt_horizontal')

cnt_horizontal
2.0     90
1.0     45
3.0     44
4.0     29
6.0     19
5.0     14
7.0      6
8.0      2
0.0      1
9.0      1
11.0     1
dtype: int64

In [893]:
hor_optimized_svod = horizontal_optimization(new_svod)

hor_optimized_svod.value_counts('cnt_horizontal')

cnt_horizontal
0.0     75
1.0     59
2.0     51
3.0     25
4.0     21
6.0      9
5.0      7
7.0      3
9.0      1
11.0     1
dtype: int64

In [894]:
vert_optimized_svod = vertical_optimization(hor_optimized_svod, new_svod)

vert_optimized_svod.value_counts('cnt_horizontal')

cnt_horizontal
2.0    120
1.0     83
3.0     41
0.0      7
4.0      1
dtype: int64

In [898]:
optimized_svod = vert_optimized_svod.reset_index()
optimized_svod['InLists'] = optimized_svod['cnt_horizontal']
optimized_svod = optimized_svod.drop('cnt_horizontal', axis=1).sort_values('Рубрика')

In [899]:
final_svod = original_svod.sort_values('Рубрика')

In [900]:
for col in optimized_svod.columns:
    final_svod[col] = optimized_svod[col]
final_svod = final_svod.sort_values('Рубрика')

In [903]:
final_svod['inList, сумма'] = final_svod[svod.columns].sort_values('Рубрика').drop(['InLists', 'inList, сумма', 'Рубрика'], axis=1).sum(axis=1)

In [910]:
final_svod['inList, среднее'] = final_svod[svod.columns].sort_values('Рубрика').drop(['InLists', 'inList, сумма', 'Рубрика'], axis=1).mean(axis=1)

In [912]:
final_svod.to_excel('final_svod.xlsx')